# Forecasting Air Quality in Beijing
Capstone Project for Springboard Data Science Bootcamp

Tamara Monge

## 1. The Project
Beijing’s air quality has been an enormous health concern in recent decades as the city has become further industrialized and more and more of its citizens have begun driving automobiles.  Over recent years, the city has improved its air quality significantly. However, those living in Beijing must remain vigilant to air quality forecasts. 

This project explores patterns in Beijing's PM 2.5 (particulate matter 2.5 microns or less in diameter) concentration and forecasts future concentrations using both classical statistical modeling and several machine learning models. 

## 2. The Clients
The client in this study is the people of Beijing. PM 2.5 is especially dangerous because it can pass through the human body’s natural filters and enter the lungs. Health concerns related to PM 2.5 include heart and lung disease, asthma, bronchitis, and other respiratory problems. The people of Beijing will be interested in having a fast, accurate, and computationally inexpensive method for forecasting the PM 2.5 concentration.

## 3. The Data: source and description 
The data used for this project comes from UCI’s [Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data "UCI Machine Learning Repository"). It is available for free download. 

The dataset consists of hourly observations between January 1, 2010 and December 31, 2014, for a total of 43,824 observations (~2 MB). Each observation contains 12 features: year, month, day, hour, PM 2.5 concentration, temperature, dewpoint temperature, pressure, combined wind direction, cumulated wind speed, cumulated hours of snow, and cumulated hours of rain. 

## 4. Wrangling 
The data came in the form of a CSV file which allowed it to be imported as pandas dataframe. To make data operations convenient, features were renamed. The `year`, `month`, `day`, and `hour` features were combined into one feature, `date`. This feature was then assigned as the dataframe's index to assist with analyses and plotting. 

Approximately 4.7% of the records were missing `pm25` values. Since the missing data was such a small percentage, I decided it was reasonable to simply drop those observations. This choice brought the number of observations down to 41,757. The table below shows the cleaned features. 

Cleaned Field | Description 
:---|:---
`date` | Datetime of observation 
`pm25` | PM 2.5 concentration ($\mu$g/m$^3$) 
`dewp` | Dewpoint temperature ($^\circ$C) 
`temp` | Temperature ($^\circ$C) 
`pres` | Pressure (hPa) 
`wind_dir` | Combined wind direction ($^\circ$) 
`wind_speed` | Cummulated wind speed (m/s)
`snow_hours` | Cummulated hours of snow
`rain_hours` | Cummulated hours of rain 


## 5. Exploratory Data Analysis
The original time series for each feature is shown in the figure below.   ![title](figures/all_time_series.png)

As expected, all seven features show strong seasonality. The seasonality of PM 2.5, pressure, wind speed, and hours of snow are nearly in alignment, while the seasonality of temperature, dewpoint, and hours of rain are offset from the others by approximately half a phase (6 months). 

The correlation between PM 2.5 and the other time series are shown below.  

Field | Correlation Coefficient 
:---|:--:
`dewp` | 0.171423 
`temp` | -0.090534
`pres` |  -0.047282
`wind_speed` | -0.247784
`snow_hours` |0.019266
`rain_hours` |  -0.051369

At first glance it appears that PM 2.5 is most strongly correlated with wind speed (-0.25) and dewpoint (0.17). However, one must keep in mind that these correlations combine seasonal and trend effects. The correlation coefficients after trends have been removed (by way of first-order differencing) are shown below. There, we see that the time series most strongly correlated with PM 2.5 is the dewpoint (0.15). 

Field | Correlation Coefficient 
:---|:--:
`dewp` | 0.152631
`temp` | -0.072160
`pres` |  -0.018048
`wind_speed` | -0.028696
`snow_hours` |-0.003552
`rain_hours` |  -0.015152


## 6. Time Series Analysis

First, let's look at the autocorrelation and partial autocorrelation functions of the original PM 2.5 time series. ![title](figures/acf_oneweek.png) ![title](figures/acf_oneday.png) The ACF shows a gradual decay, with significant autocorrelations out to ~82 hours.
![title](figures/pacf_oneweek.png) ![title](figures/pacf_oneday.png)
The PACF shows that most of that autocorrelation can be explained by the first 2 lags.

Examining the decomposed time series (figure below) reveals only little trend and seasonality, suggesting the time series is stationary.  ![title](figures/seasonal_decomposition_daily.png)

We used the Augmented Dickey Fuller to test for stationarity. The test statistic was -20.6, well below the 1% critical value of -3.4. This means we can say with 99% confidence that the PM 2.5 time series is already stationary and thus there is no need to apply difference or log transforms. This stationarity is seen visually in the averaged values shown below. ![title](figures/stationary_series_check.png)

## 7. Forecasting Models
### 7.1 Model selection 
Eight models were fit for this project: a classical statistical forecasting model (seasonal ARIMA), an additive regression model (Prophet), and six supervised learning regressors. 

#### (A) Seasonal Autoregressive Integrated Moving Average Model: SARIMA
The classical statistical modeling technique for time series is a the autoregressive integrated moving average model (ARIMA). When a seasonal component is observed in the data, the seasonal ARIMA (SARIMA) is used. Since this is the case for our time series, we started our modeling efforts with the SARIMA. 

We performed a gridsearch with the AIC as the scoring metric to obtain the best parameters for the SARIMA: (2, 0, 1)x(2, 1, 1, 24). The results of the model (below) indicate that there the most important component is the autoregression at lag-1. However, all components are significant, including the seasonal components. While the plots of the residuals (upper left), quantiles (lower left) and autocorrelation (lower right) show strong performance, the histogram of the residuals (upper right) indicates the residuals are not quite normally distributed, and thus the model performance is not ideal. ![title](figures/SARIMA_results_table.png)
![title](figures/sarima_figures.png)

#### (B) Additive regression model: Prophet
Prophet is a nonparametric regression model that works very well with hourly observations over a long period of time. The results of the prophet model (below) reveal a significant daily, weekly, and annual component to the PM 2.5 concentration.  ![title](figures/prophet_forecast_components.png)
As seen in the figure below, the prophet model's forecast on the last year of data is very accurate.
![title](figures/prophet_forecast.png) 

#### (C) Supervised learning regressors
Time series forecasting can also be cast as a supervised learning regression problem where the target variable ($y$) is the feature `pm25` and the predictive variables ($X$) are ($y$) at various lags. A significant benefit to using this approach is the ability to incoporate multivariate time series into the predictive variable. For this project we use `pm25` at hourly lags up to one week, and `dewp` at the same lags as our predictive variables. We have chosen the following six regressors for this problem: 

##### Linear Regression
The benefit of linear regression is that it is the simplest regressor and is readily interpretable. 

##### K-Nearest Neighbor
K-Nearest neighbors is also a simple and easily interpretable model.

#####  Stochastic Gradient Descent
The benefit of stochastic gradient descent regressors it that they work well with a large number of training samples. 

#####  Decision Tree
Decision tree regressors are computationally fast and highly interpretable. One drawback to decision trees is that they are prone to overfitting.  

#####  Random Forest 
Random forest regressors are an ensemble of decision tree regressors. As a result, they carry some of the advantages of decision trees, namely they are computationally fast and work well with high dimensions. At the same time the ensemble nature of random forests reduces overfitting, thereby addressing the biggest disadvantage of decision trees. 

#####  Multi-layer Perceptron (neural network)
A benefit of multi-layer perceptrons is that they can be configured to learn models in real time thereby incorporating new observations. 

### 7.2 Data splitting 
The last year of observations was held out as a test set. 

For the supervised learning models, the remaining observations were randomly split into training and evaluation sets (70% and 30% of the remaining observations, respectively). The training set was used to fit the models and the evaluation set was used to evaluate the fitted models and determine the best-performing model (Section 7.3). 

The test set was held out until the final test stage (Section 7.4) to get an estimate of how well the best-performing algorithm generalizes to unseen data.

### 7.3 Model evaluation 
AIC measures trade-off between goodness of fit of the model and the complexity of the model. 

The performance of each fitted regressor was assessed by predicting on the evaluation set.

As shown in the table below, most of the regressors out-performed the Seasonal ARIMA and Prophet models in R$^2$ score and Mean Square Error. The **multi-layer perceptron (neural network)** just edged out the random forest.


Model | R$^2$ | MSE  
--- | :---: | :---: |
Multi-layer Perceptron | 0.998 | 18.3 
Random Forest | 0.994 | 50.5 
Decision Tree | 0.984 | 130
KNN Regression | 0.973 | 227 
Linear Regression | 0.931 | 578 
Prophet | -.026 | 6.83e+3
Stochastic Gradient Descent | -2.86e+16 | 2.39e+20 

### 7.4 Model results
To evaluate the performance of the fitted multi-layer perceptron on un-seen data, it was tested against the held-out test set (see Section 7.2 for a description of the test set). The results are shown below. ![title](figures/forecast_results.png)

Metric | Multi-Layer Perceptron Test Score
---:|:---:
R$^2$ |0.954
MSE ($\mu$g/m$^3$) |414 
Forecast Bias ($\mu$g/m$^3$) | -1.94 



**The multi-layer perceptron algorithm captures 95.4% of the variance of out-of-sample PM 2.5 concentrations**. This is excellent for a statistical (non-physics-based) model. While the model tends to underforecast the PM 2.5 concentration by 1.9 $\mu$g/m$^3$, this is a mere **2% bias** when compared to the average concentration reported in Beijing (98.6 $\mu$g/m$^3$).



## 8. Limitations
- It is possible that there was an unknown pattern (e.g., very high PM 2.5 concentrations) to the records that did not contain PM 2.5 information, such that the exclusion of those records may have introduced bias into the results.

## 9. Further research
- Increasing the number of lag features included in the supervised learning models may improve performance.

- Adding more variables (such as temperature, pressure, wind speed, etc.) to the supervised learning models may further improve performance. 

- Adding exogenous variables (such as day-of-week) to the SARIMA model may improve that model's performance. 

## 10. Client recommendation
- For fast, accurate forecasting that is less computationally expensive than traditional physics-based air quality models, the client should deploy the multi-layer-perceptron (neural network) model developed here. In addition to providing high-quality forecasts, it can also quickly learn on new observations. 

### Acknowlegements

I would like to thank Springboard. I would also like to thank my mentor Amir Ziai for his guidance and suggestions. I also thank UCI for making machine learning datasets openly available. 

---------
__Copyright 2018 Tamara Monge__